In [10]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import csv
import requests
import pandas as pd
import re
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities #다음화면으로 빨리 넘어갈때
from selenium.common.exceptions import UnexpectedAlertPresentException #selenium 이용시 Alert을 제어
import urllib.request
import random
from selenium.webdriver.chrome.options import Options

In [11]:
# 크롤링 시작과 끝 날짜(달 별로 크롤링)
first_days = pd.date_range('2019/01/01','2019/12/31', freq='MS')   # 각 달의 첫 날
last_days = pd.date_range('2019/01/01','2019/12/31', freq='M') 


keyword_list =  ['인스타술집','이색술집','감성술집', '컨셉술집','분위기술집']    # 검색어 키워드 추가

In [12]:
# 크롬드라이버 제어
chrome_options = Options()
chrome_options.add_argument('--no-sandbox') #bypass OS security model
chrome_options.add_argument('--disable-dex-shm-usage')#overcome limited resource problems
# options.add_argument('--headless') # 브라우저 백그라운드 모드


chrome_path = r'C:\Users\user\Downloads\chromedriver_win32 (1)\chromedriver.exe' #현재 컴퓨터의 크롬드라이버 위치

In [13]:
# 블로그 주소 수집

blog_url_list = []

#네이버에 키워드 검색
for keyword in keyword_list:
    url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=blog&query={}&oquery={}'.format(keyword, keyword)
        
    driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)
    driver.implicitly_wait(3)
    
    driver.get(url)
    time.sleep(3)
    
    url_want = 990 #
    
    driver.find_element('xpath','//*[@id="snb"]/div[1]/div/div[2]/a').click() #검색 옵션 클릭

    #날짜 지정
    for k in range(len(first_days)):
        try:
            #시작과 끝의 년월일 설정
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]/i').click() #직접 입력
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(first_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(first_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(first_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[3]/a').click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(last_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(last_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(last_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[3]/button').click()            
            
            
            # 스크롤 조작
            last_height = driver.execute_script('return document.body.scrollHeight')
            
            scroll = (url_want/30)-1 #한 페이지에 기본으로 30개 글이 나옴
            
            # 스크롤 다운
            for i in range(int(scroll)):
                driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
                time.sleep(random.uniform(1,1.7))
                
                new_height = driver.execute_script('return document.body.scrollHeight')
                
                if new_height == last_height:
                    break
                    
                last_height = new_height
                
                
            soup = bs(driver.page_source, 'lxml')
            blog_url = soup.find_all('a',class_='api_txt_lines total_tit')
            
            # url 주소 수집
            for i in blog_url:
                blog_url_list.append(i['href']) #주소 수집
                
            print('{0} 키워드 {1}년{2}월'.format(keyword, 
                                            first_days.year[k], frist_days.month[k]), 
                  len(blog_url),'개 블로그 url 주소 수집')
            
            
            driver.execute_script('window.scrollTo(0,0)')
            
        except:
            pass
            

C:\Users\user\AppData\Local\Temp\ipykernel_3656\707424404.py:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)


In [14]:
k = pd.DataFrame(blog_url_list)
k.to_csv('서울컨셉술집2.csv',encoding = 'utf-8-sig' )

In [19]:
url = pd.read_csv('서울컨셉술집2.csv')

blog_url_list= list(url['0'])
len(blog_url_list)

21390

In [20]:
no_naver_blog1=[]

start = time.time()


time_list = [] #시간
review_list = [] #리뷰
comment_list = [] #댓글
like_list = [] #좋아요수
url_list = [] #url주소
count = 0

driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options) #크롬 드라이버 위치만 바꿔주시면 됩니다.
for url in blog_url_list: #기본적으로 고치지 않으시지만, n번째 크롤링에서 멈췄다면, blog_url_list를 blog_url_list[n:]으로 고칩니다.
    if 'naver' in url: #네이버 블로그만 수집(다른 사이트가 섞이면 크롤링 에러가 남)
    
        count += 1
        driver.get(url) #url 하나씩 글을 수집할꺼야
        time.sleep(1)

        try:
            # 전체 본문 가지고 와서
            driver.switch_to.frame('mainFrame') #네이버는 mainFrame을 바꿔줘야 수집이 됌
            time.sleep(1)
        
            soup = bs(driver.page_source, 'lxml') #뷰티풀숩을 이용해서 구문 분석(lxml 파싱 이용. 모든 테그에서 데이터를 추출위함)
            postview = soup.find('div', attrs={'id': re.compile('post-view.+')}).get_text()

            reg = re.compile(r'[\s+]') #r은 raw string. \s+는 공백문자
            postview_reg = reg.sub(' ',postview) #공백을 띄어쓰기로 대치해서 본문 글 수집
            try:
                # 좋아요수
                like = driver.find_element("xpath", "//span[@class='u_likeit_list_btn _button btn_sympathy pcol2 off']/em[@class='u_cnt _count']")
                like = like.text
            except:
                like = [] #좋아요 없으면 빈 리스트
            
            try:
                timeline = driver.find_element("xpath", "//span[@class='se_publishDate pcol2']") #수정 안 한 경우
        
            except:
                timeline = driver.find_element("xpath", "//p[@class='date fil5 pcol2 _postAddDate']") #수정 한 경우
            timeline = timeline.text

            try:
                driver.find_element(By.CSS_SELECTOR,'span.btn_arr').click() #댓글 내리는 버튼(태그명.클래스선택자)
                #driver.find_element_by_css_selector('span.btn_arr').click()
                time.sleep(1.7)

                comment_blog = []
                comment = driver.find_element(By.CSS_SELECTOR,'span.u_cbox_contents') #댓글
                #comment = driver.find_elements_by_css_selector('span.u_cbox_contents')
                for i in comment: #댓글 수집
                    com = str(i.text) #
                    com = reg.sub(' ',com) 
                    comment_blog.append(([com]))
            except: 
                comment_blog = []
             
            time_list.append(timeline)
            review_list.append(postview_reg)
            comment_list.append(comment_blog)
            url_list.append(url)
            like_list.append(like)
            time.sleep(random.uniform(1,1.6))
        except UnexpectedAlertPresentException:
            pass
    else:
        no_naver_blog1.append(url)
        print('네이버 블로그 아닌 url: ',url)
        
    if count%10 ==0:
        print('{}개 블로그 크롤링 완료'.format(count))

C:\Users\user\AppData\Local\Temp\ipykernel_3656\3172629000.py:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options) #크롬 드라이버 위치만 바꿔주시면 됩니다.


10개 블로그 크롤링 완료
20개 블로그 크롤링 완료
30개 블로그 크롤링 완료
40개 블로그 크롤링 완료
50개 블로그 크롤링 완료
60개 블로그 크롤링 완료
70개 블로그 크롤링 완료
80개 블로그 크롤링 완료
90개 블로그 크롤링 완료
100개 블로그 크롤링 완료
110개 블로그 크롤링 완료
120개 블로그 크롤링 완료
130개 블로그 크롤링 완료
140개 블로그 크롤링 완료
150개 블로그 크롤링 완료
160개 블로그 크롤링 완료
170개 블로그 크롤링 완료
180개 블로그 크롤링 완료
190개 블로그 크롤링 완료
200개 블로그 크롤링 완료
210개 블로그 크롤링 완료
220개 블로그 크롤링 완료
230개 블로그 크롤링 완료
240개 블로그 크롤링 완료
250개 블로그 크롤링 완료
260개 블로그 크롤링 완료
270개 블로그 크롤링 완료
280개 블로그 크롤링 완료
290개 블로그 크롤링 완료
300개 블로그 크롤링 완료
310개 블로그 크롤링 완료
320개 블로그 크롤링 완료
330개 블로그 크롤링 완료
340개 블로그 크롤링 완료
350개 블로그 크롤링 완료
360개 블로그 크롤링 완료
370개 블로그 크롤링 완료
380개 블로그 크롤링 완료
390개 블로그 크롤링 완료
400개 블로그 크롤링 완료
410개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://toygrim.tistory.com/621
420개 블로그 크롤링 완료
430개 블로그 크롤링 완료
440개 블로그 크롤링 완료
450개 블로그 크롤링 완료
460개 블로그 크롤링 완료
470개 블로그 크롤링 완료
480개 블로그 크롤링 완료
490개 블로그 크롤링 완료
500개 블로그 크롤링 완료
510개 블로그 크롤링 완료
520개 블로그 크롤링 완료
530개 블로그 크롤링 완료
540개 블로그 크롤링 완료
550개 블로그 크롤링 완료
560개 블로그 크롤링 완료
570개 블로그 크롤링 완료
580개 블로그 크롤링 완료
590개 블로그 크롤링 완료
600개 블로그 크롤링 완료


4090개 블로그 크롤링 완료
4100개 블로그 크롤링 완료
4110개 블로그 크롤링 완료
4120개 블로그 크롤링 완료
4130개 블로그 크롤링 완료
4140개 블로그 크롤링 완료
4150개 블로그 크롤링 완료
4160개 블로그 크롤링 완료
4170개 블로그 크롤링 완료
4180개 블로그 크롤링 완료
4190개 블로그 크롤링 완료
4200개 블로그 크롤링 완료
4210개 블로그 크롤링 완료
4220개 블로그 크롤링 완료
4230개 블로그 크롤링 완료
4240개 블로그 크롤링 완료
4250개 블로그 크롤링 완료
4260개 블로그 크롤링 완료
4270개 블로그 크롤링 완료
4280개 블로그 크롤링 완료
4290개 블로그 크롤링 완료
4300개 블로그 크롤링 완료
4310개 블로그 크롤링 완료
4320개 블로그 크롤링 완료
4330개 블로그 크롤링 완료
4340개 블로그 크롤링 완료
4350개 블로그 크롤링 완료
4360개 블로그 크롤링 완료
4370개 블로그 크롤링 완료
4380개 블로그 크롤링 완료
4390개 블로그 크롤링 완료
4400개 블로그 크롤링 완료
4410개 블로그 크롤링 완료
4420개 블로그 크롤링 완료
4430개 블로그 크롤링 완료
4440개 블로그 크롤링 완료
4450개 블로그 크롤링 완료
4460개 블로그 크롤링 완료
4470개 블로그 크롤링 완료
4480개 블로그 크롤링 완료
4490개 블로그 크롤링 완료
4500개 블로그 크롤링 완료
4510개 블로그 크롤링 완료
4520개 블로그 크롤링 완료
4530개 블로그 크롤링 완료
4540개 블로그 크롤링 완료
4550개 블로그 크롤링 완료
4560개 블로그 크롤링 완료
4570개 블로그 크롤링 완료
4580개 블로그 크롤링 완료
4590개 블로그 크롤링 완료
4600개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://www.beer2day.com/2897?category=190994
4610개 블로그 크롤링 완료
4620개 블로그 크롤링 완료
4630개 블로그 크롤링 완료
46

8510개 블로그 크롤링 완료
8520개 블로그 크롤링 완료
8530개 블로그 크롤링 완료
8540개 블로그 크롤링 완료
8550개 블로그 크롤링 완료
8560개 블로그 크롤링 완료
8570개 블로그 크롤링 완료
8580개 블로그 크롤링 완료
8590개 블로그 크롤링 완료
8600개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://soulboss.tistory.com/363
8610개 블로그 크롤링 완료
8620개 블로그 크롤링 완료
8630개 블로그 크롤링 완료
8640개 블로그 크롤링 완료
8650개 블로그 크롤링 완료
8660개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://bwolf08.tistory.com/816
8670개 블로그 크롤링 완료
8680개 블로그 크롤링 완료
8690개 블로그 크롤링 완료
8700개 블로그 크롤링 완료
8710개 블로그 크롤링 완료
8720개 블로그 크롤링 완료
8730개 블로그 크롤링 완료
8740개 블로그 크롤링 완료
8750개 블로그 크롤링 완료
8760개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://hj-review.tistory.com/18
8770개 블로그 크롤링 완료
8780개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://hj-review.tistory.com/20
8790개 블로그 크롤링 완료
8800개 블로그 크롤링 완료
8810개 블로그 크롤링 완료
8820개 블로그 크롤링 완료
8830개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://fulltax.tistory.com/entry/%EC%A2%85%EB%A1%9C-%EB%8F%84%EC%BF%84%EC%88%A0%EC%A7%91-%EC%9D%B4%EC%9E%90%EC%B9%B4%EC%95%BC-%EC%86%94%EC%A7%81-%ED%9B%84%EA%B8%B0
8840개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://db-writing.tisto

12530개 블로그 크롤링 완료
12540개 블로그 크롤링 완료
12550개 블로그 크롤링 완료
12560개 블로그 크롤링 완료
12570개 블로그 크롤링 완료
12580개 블로그 크롤링 완료
12590개 블로그 크롤링 완료
12600개 블로그 크롤링 완료
12610개 블로그 크롤링 완료
12620개 블로그 크롤링 완료
12630개 블로그 크롤링 완료
12640개 블로그 크롤링 완료
12650개 블로그 크롤링 완료
12660개 블로그 크롤링 완료
12670개 블로그 크롤링 완료
12680개 블로그 크롤링 완료
12690개 블로그 크롤링 완료
12700개 블로그 크롤링 완료
12710개 블로그 크롤링 완료
12720개 블로그 크롤링 완료
12730개 블로그 크롤링 완료
12740개 블로그 크롤링 완료
12750개 블로그 크롤링 완료
12760개 블로그 크롤링 완료
12770개 블로그 크롤링 완료
12780개 블로그 크롤링 완료
12790개 블로그 크롤링 완료
12800개 블로그 크롤링 완료
12810개 블로그 크롤링 완료
12820개 블로그 크롤링 완료
12830개 블로그 크롤링 완료
12840개 블로그 크롤링 완료
12850개 블로그 크롤링 완료
12860개 블로그 크롤링 완료
12870개 블로그 크롤링 완료
12880개 블로그 크롤링 완료
12890개 블로그 크롤링 완료
12900개 블로그 크롤링 완료
12910개 블로그 크롤링 완료
12920개 블로그 크롤링 완료
12930개 블로그 크롤링 완료
12940개 블로그 크롤링 완료
12950개 블로그 크롤링 완료
12960개 블로그 크롤링 완료
12970개 블로그 크롤링 완료
12980개 블로그 크롤링 완료
12990개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://xaxo.tistory.com/2060
13000개 블로그 크롤링 완료
13010개 블로그 크롤링 완료
13020개 블로그 크롤링 완료
13030개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://bwolf08.ti

15620개 블로그 크롤링 완료
15630개 블로그 크롤링 완료
15640개 블로그 크롤링 완료
15650개 블로그 크롤링 완료
15660개 블로그 크롤링 완료
15670개 블로그 크롤링 완료
15680개 블로그 크롤링 완료
15690개 블로그 크롤링 완료
15700개 블로그 크롤링 완료
15710개 블로그 크롤링 완료
15720개 블로그 크롤링 완료
15730개 블로그 크롤링 완료
15740개 블로그 크롤링 완료
15750개 블로그 크롤링 완료
15760개 블로그 크롤링 완료
15770개 블로그 크롤링 완료
15780개 블로그 크롤링 완료
15790개 블로그 크롤링 완료
15800개 블로그 크롤링 완료
15810개 블로그 크롤링 완료
15820개 블로그 크롤링 완료
15830개 블로그 크롤링 완료
15840개 블로그 크롤링 완료
15850개 블로그 크롤링 완료
15860개 블로그 크롤링 완료
15870개 블로그 크롤링 완료
15880개 블로그 크롤링 완료
15890개 블로그 크롤링 완료
15900개 블로그 크롤링 완료
15910개 블로그 크롤링 완료
15920개 블로그 크롤링 완료
15930개 블로그 크롤링 완료
15940개 블로그 크롤링 완료
15950개 블로그 크롤링 완료
15960개 블로그 크롤링 완료
15970개 블로그 크롤링 완료
15980개 블로그 크롤링 완료
15990개 블로그 크롤링 완료
16000개 블로그 크롤링 완료
16010개 블로그 크롤링 완료
16020개 블로그 크롤링 완료
16030개 블로그 크롤링 완료
16040개 블로그 크롤링 완료
16050개 블로그 크롤링 완료
16060개 블로그 크롤링 완료
16070개 블로그 크롤링 완료
16080개 블로그 크롤링 완료
16090개 블로그 크롤링 완료
16100개 블로그 크롤링 완료
16110개 블로그 크롤링 완료
16120개 블로그 크롤링 완료
16130개 블로그 크롤링 완료
16140개 블로그 크롤링 완료
16150개 블로그 크롤링 완료
16160개 블로그 크롤링 완료
16170개 블로그

19150개 블로그 크롤링 완료
19160개 블로그 크롤링 완료
19170개 블로그 크롤링 완료
19180개 블로그 크롤링 완료
19190개 블로그 크롤링 완료
19200개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://finalmonster.tistory.com/263
19210개 블로그 크롤링 완료
19220개 블로그 크롤링 완료
19230개 블로그 크롤링 완료
19240개 블로그 크롤링 완료
19250개 블로그 크롤링 완료
19260개 블로그 크롤링 완료
19270개 블로그 크롤링 완료
19280개 블로그 크롤링 완료
19290개 블로그 크롤링 완료
19300개 블로그 크롤링 완료
19310개 블로그 크롤링 완료
19320개 블로그 크롤링 완료
19330개 블로그 크롤링 완료
19340개 블로그 크롤링 완료
19350개 블로그 크롤링 완료
19360개 블로그 크롤링 완료
19370개 블로그 크롤링 완료
19380개 블로그 크롤링 완료
19390개 블로그 크롤링 완료
19400개 블로그 크롤링 완료
19410개 블로그 크롤링 완료
19420개 블로그 크롤링 완료
19430개 블로그 크롤링 완료
19440개 블로그 크롤링 완료
19450개 블로그 크롤링 완료
19460개 블로그 크롤링 완료
19470개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://realtaeheon.tistory.com/1308
19480개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://xaxo.tistory.com/2092
19490개 블로그 크롤링 완료
19500개 블로그 크롤링 완료
19510개 블로그 크롤링 완료
19520개 블로그 크롤링 완료
19530개 블로그 크롤링 완료
19540개 블로그 크롤링 완료
19550개 블로그 크롤링 완료
19560개 블로그 크롤링 완료
19570개 블로그 크롤링 완료
19580개 블로그 크롤링 완료
19590개 블로그 크롤링 완료
19600개 블로그 크롤링 완료
19610개 블로그 크롤링 완료

In [21]:
data = {'time': time_list,
       'review': review_list,
       'comment':comment_list,
       'like':like_list,
       'url':url_list}

df = pd.DataFrame(data)
print('크롤링 소요시간', time.time()-start)
print('{}개 블로그 크롤링 완료'.format(count))

크롤링 소요시간 103895.06552362442
21246개 블로그 크롤링 완료


In [22]:
#ccv  저장
df.to_csv("2019술집.csv", encoding = 'utf-8-sig')